In [2]:
import pandas as pd
import kagglehub
import os

In [3]:
# Disable SSL verification (not recommended for production)
os.environ["KAGGLE_HTTP_NO_VERIFY"] = "1"
os.environ["SSL_CERT_FILE"] = ""

import requests
from requests.packages.urllib3.exceptions import InsecureRequestWarning

# Suppress only the single InsecureRequestWarning from urllib3 needed.
requests.packages.urllib3.disable_warnings(category=InsecureRequestWarning)

# Patch requests to disable SSL verification globally
old_request = requests.Session.request
def new_request(self, *args, **kwargs):
	kwargs['verify'] = False
	return old_request(self, *args, **kwargs)
requests.Session.request = new_request

# Download latest version
path = kagglehub.dataset_download("rehandl23/fifa-24-player-stats-dataset")
print("Path to dataset files:", path)

100%|██████████| 357k/357k [00:00<00:00, 2.16MB/s]

Extracting files...
Path to dataset files: /home/vscode/.cache/kagglehub/datasets/rehandl23/fifa-24-player-stats-dataset/versions/2


In [4]:
df = pd.read_csv(rf"{path}/player_stats.csv", encoding="latin1")
df.dtypes

player             object
country            object
height              int64
weight              int64
age                 int64
club               object
ball_control        int64
dribbling           int64
marking           float64
slide_tackle        int64
stand_tackle        int64
aggression          int64
reactions           int64
att_position        int64
interceptions       int64
vision              int64
composure           int64
crossing            int64
short_pass          int64
long_pass           int64
acceleration        int64
stamina             int64
strength            int64
balance             int64
sprint_speed        int64
agility             int64
jumping             int64
heading             int64
shot_power          int64
finishing           int64
long_shots          int64
curve               int64
fk_acc              int64
penalties           int64
volleys             int64
gk_positioning      int64
gk_diving           int64
gk_handling         int64
gk_kicking  

In [ ]:
df.head()[["dribbling", "aggression", "penalties"]]

,dribbling,aggression,penalties
0,43,72,41
1,70,62,48
2,12,27,26
3,48,71,22
4,8,28,16


In [5]:
import plotly.express as px

# Convertir la columna 'value' a numérico (eliminar símbolos y convertir a int)
def parse_value(val):
    if isinstance(val, str):
        val = val.replace('$', '').replace('.', '').replace(',', '')
        try:
            return int(val)
        except ValueError:
            return 0
    return 0

df['value_eur'] = df['value'].apply(parse_value)

# Filtrar jugadores colombianos y seleccionar el top 10 por valor
top_colombian_players = df[df['country'] == 'Colombia'].nlargest(10, 'value_eur')

fig = px.bar(
    top_colombian_players,
    x='value_eur',
    y='player',
    orientation='h',
    labels={'value_eur': 'Valor en Euros', 'player': 'Jugador'},
    title='Top 10 jugadores colombianos más costosos'
)
fig.update_layout(yaxis={'categoryorder':'total ascending'})
fig.show()